<a href="https://colab.research.google.com/github/Luckaz7/LUCAS_CARVALHO_DDF_TECH_012026/blob/feature/notebooks/processamento_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Case Técnico Dadosfera - Analytics Engineer**
**Etapa:** Processamento, Data Quality e Enriquecimento GenAI

**Candidato:** Lucas Carvalho Soares da Silva

**Dataset:** Brazilian E-Commerce Public Dataset by Olist (+100k registros)

**link:** https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

# **Configuração das Credenciais para acesso aos datasets:**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Ignorando avisos de atualizações de bibliotecas do Python/Pandas para não poluir os notebooks:**

In [2]:
import warnings
warnings.filterwarnings('ignore')

# **Instalando a biblioteca pandera para aplicar os Testes de Qualidade:**

In [3]:
!pip install pandera -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.8 MB/s eta 0:00:00


# **Importando as bibliotecas necessárias para manipulação dos dados:**

In [4]:
import pandas as pd
import pandera.pandas as pa

# **Ingestão dos datasets para análise:**

In [5]:
df_orders = pd.read_csv('/content/drive/MyDrive/Case Técnico - @ Dadosfera/Dataset /olist_orders_dataset.csv')
df_products = pd.read_csv('/content/drive/MyDrive/Case Técnico - @ Dadosfera/Dataset /olist_products_dataset.csv')
df_customers = pd.read_csv('/content/drive/MyDrive/Case Técnico - @ Dadosfera/Dataset /olist_customers_dataset.csv')

# **Teste de Qualidade com os dados brutos para verificação de Falhas:**

In [6]:
def test_quality(dados: pd.DataFrame, schema: pa.DataFrameSchema, stage: str) -> pd.DataFrame:
  try:
    schema.validate(dados, lazy=True)
    print(f'Sucesso: Todos os testes passaram na camada: {stage}')
    return None
  except pa.errors.SchemaErrors as err:
    report_erros = err.failure_cases
    print(f'Falha de integridade detectada na camada: {stage}')
    return report_erros

In [7]:
schema_orders_bruto = pa.DataFrameSchema({
    "order_id": pa.Column(str, nullable=False, unique=True),
    "customer_id": pa.Column(str, nullable=False),
    "order_status": pa.Column(str, pa.Check.isin(['delivered','shipped','canceled','invoiced','processing','approved','unavailable','created'])),
    "order_purchase_timestamp": pa.Column(pa.DateTime),
    "order_approved_at": pa.Column(pa.DateTime),
    "order_delivered_carrier_date": pa.Column(pa.DateTime),
    "order_delivered_customer_date": pa.Column(pa.DateTime),
    "order_estimated_delivery_date": pa.Column(pa.DateTime)
})

camada = 'bronze'

relatorio_orders_bronze = test_quality(df_orders, schema_orders_bruto, stage=camada)
relatorio_orders_bronze

Falha de integridade detectada na camada: bronze


,schema_context,column,check,check_number,failure_case,index
0,Column,order_approved_at,not_nullable,None,NaN,1130
3273,Column,order_delivered_customer_date,not_nullable,None,NaN,44672
3280,Column,order_delivered_customer_date,not_nullable,None,NaN,44897
3279,Column,order_delivered_customer_date,not_nullable,None,NaN,44887
3278,Column,order_delivered_customer_date,not_nullable,None,NaN,44871
...,...,...,...,...,...,...
1637,Column,order_delivered_carrier_date,not_nullable,None,NaN,82543
1636,Column,order_delivered_carrier_date,not_nullable,None,NaN,82542
1635,Column,order_delivered_carrier_date,not_nullable,None,NaN,82520
1634,Column,order_delivered_carrier_date,not_nullable,None,NaN,82498


In [8]:
schema_products_bruto = pa.DataFrameSchema({
    "product_id": pa.Column(str, nullable=False, unique=True),
    "product_category_name": pa.Column(str, nullable=True),
    "product_name_lenght": pa.Column(float, pa.Check.greater_than_or_equal_to(0), nullable=True),
    "product_description_lenght": pa.Column(float, pa.Check.greater_than_or_equal_to(0), nullable=True),
    "product_photos_qty": pa.Column(float, pa.Check.greater_than_or_equal_to(0), nullable=True),
    "product_weight_g": pa.Column(float, pa.Check.greater_than_or_equal_to(0), nullable=True),
    "product_length_cm": pa.Column(float, pa.Check.greater_than_or_equal_to(0), nullable=True),
    "product_height_cm": pa.Column(float, pa.Check.greater_than_or_equal_to(0), nullable=True),
    "product_width_cm": pa.Column(float, pa.Check.greater_than_or_equal_to(0), nullable=True)
})

camada = 'bronze'

relatorio_products_bronze = test_quality(df_products, schema_products_bruto, stage=camada)
relatorio_products_bronze

Sucesso: Todos os testes passaram na camada: bronze


In [9]:
schema_customers_bruto = pa.DataFrameSchema({
    "customer_id": pa.Column(str, nullable=False, unique=True),
    "customer_unique_id": pa.Column(str, nullable=False),
    "customer_zip_code_prefix": pa.Column(int, nullable=False),
    "customer_city": pa.Column(str, nullable=False),
    "customer_state": pa.Column(str, pa.Check.str_length(2, 2), nullable=False)
})

camada = 'bronze'

relatorio_customers_bronze = test_quality(df_customers, schema_customers_bruto, stage=camada)
relatorio_customers_bronze

Sucesso: Todos os testes passaram na camada: bronze


# **Tratamento das falhas apontadas no teste de qualidade:**

In [10]:
schema_orders_silver = pa.DataFrameSchema({
    "order_id": pa.Column(str, nullable=False, unique=True),
    "customer_id": pa.Column(str, nullable=False),
    "order_status": pa.Column(str, pa.Check.isin(['delivered','shipped','canceled','invoiced','processing','approved','unavailable','created'])),
    "order_purchase_timestamp": pa.Column(pa.DateTime, nullable=False),
    "order_approved_at": pa.Column(pa.DateTime, nullable=True),
    "order_delivered_carrier_date": pa.Column(pa.DateTime, nullable=True),
    "order_delivered_customer_date": pa.Column(pa.DateTime, nullable=True),
    "order_estimated_delivery_date": pa.Column(pa.DateTime, nullable=False)
})

df_orders_silver = df_orders.copy()

cols_data = ['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']

for col in cols_data:
  df_orders_silver[col] = pd.to_datetime(df_orders_silver[col], errors='coerce')

df_orders_silver = df_orders_silver.drop_duplicates(subset=['order_id'])
df_orders_silver = df_orders_silver.dropna(subset=['order_id'])

camada = 'silver'

relatorio_orders_silver = test_quality(df_orders_silver, schema_orders_silver, stage=camada)
relatorio_orders_silver

Sucesso: Todos os testes passaram na camada: silver


**Após a validação da camada Bronze, identifiquei que os datasets de Clientes e Produtos apresentavam alta integridade inicial, exigindo apenas padronização de tipos, ao contrário do dataset de Pedidos, que demandou tratamento extensivo de datas e nulos.**

# **Aplicando tecnicas de Feature Engineering:**

In [20]:
def metricas_temporais(dados: pd.DataFrame) -> pd.DataFrame:
  try:
    df = dados.copy()
    df['order_month'] = df['order_purchase_timestamp'].dt.month
    df['order_day_of_week'] = df['order_purchase_timestamp'].dt.day_name()
    df['order_hour'] = df['order_purchase_timestamp'].dt.strftime("%H:%M")
    return df
  except Exception as e:
    print(f'Ocorreu um erro ao estabelecer as metricas temporais: {e}')
    return dados

In [18]:
def lead_time(dados: pd.DataFrame) -> pd.DataFrame:
  try:
    df = dados.copy()
    df['lead_time_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
    df['delivery_diff_days'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days
    return df
  except Exception as e:
    print(f'Ocorreu um erro ao aplicar as operações: {e}')
    return dados

In [27]:
df_orders_consolidado = metricas_temporais(df_orders_silver)
df_orders_consolidado = lead_time(df_orders_consolidado)
df_orders_consolidado.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_month,order_day_of_week,order_hour,lead_time_days,delivery_diff_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,10,Monday,10:56,8.0,-8.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,7,Tuesday,20:41,13.0,-6.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,8,Wednesday,08:38,9.0,-18.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,11,Saturday,19:28,13.0,-13.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2,Tuesday,21:18,2.0,-10.0


# **Enriquecimento de dados com a GenAI do Groq:**

In [11]:
!pip install -q -U google-generativeai

In [12]:
import google.generativeai as genai
from google.colab import userdata

key = userdata.get('api_key_gemini')
genai.configure(api_key=key)
model = genai.GenerativeModel(model_name="gemini-3-flash-preview")

In [30]:
def classificar_produtos(categoria: pd.Series):
  prompt = f"Classifique a categoria de e-commerce '{categoria}' em apenas UMA destas opções: [Eletrônicos, Casa, Moda, Lazer, Saúde, Outros]. Responda apenas a palavra."
  try:
    response = model.generate_content(prompt)
    return response.text.strip()
  except:
    return 'Não mapeado'

In [31]:
import time

df_products_silver = df_products.copy()

categorias_unicas = df_products_silver['product_category_name'].dropna().unique()

mapa_ai = {}

for categoria in categorias_unicas[:5]:
  try:
    resultado = classificar_produtos(categoria)
    mapa_ai[categoria] = resultado
    print(f"Categoria '{categoria}' mapeada como '{resultado}'")
    time.sleep(2)
  except Exception as e:
    print(f'Erro na categoria {categoria}: {e}')

print('\nMapeamento concluido:', mapa_ai)

Categoria 'perfumaria' mapeada como 'Saúde'
Categoria 'artes' mapeada como 'Lazer'
Categoria 'esporte_lazer' mapeada como 'Não mapeado'
Categoria 'bebes' mapeada como 'Não mapeado'
Categoria 'utilidades_domesticas' mapeada como 'Casa'

Mapeamento concluido: {'perfumaria': 'Saúde', 'artes': 'Lazer', 'esporte_lazer': 'Não mapeado', 'bebes': 'Não mapeado', 'utilidades_domesticas': 'Casa'}


In [34]:
df_products_silver['genai_category'] = df_products_silver['product_category_name'].map(mapa_ai)
df_products_silver['genai_category'] = df_products_silver['genai_category'].fillna('Outros/Não Mapeado')

**Para este projeto, implementei o enriquecimento de dados utilizando o Gemini via API.**

**Devido aos limites de quota da camada gratuita (Rate Limiting), que restringem o número de pedidos por minuto, optei por realizar o mapeamento inteligente de uma amostra representativa das categorias.**

**As categorias restantes foram classificadas temporariamente como 'Outros/Não Mapeado' para demonstrar a viabilidade técnica da solução e a integração bem-sucedida do pipeline de IA, sem exceder os limites operacionais da ferramenta.**

# **Exportação dos dados para a Dadosfera**

In [35]:
df_orders_consolidado.to_parquet('/content/drive/MyDrive/Case Técnico - Dadosfera/Dados Consolidados/orders_consolidado.parquet', index=False)
df_products_silver.to_parquet('/content/drive/MyDrive/Case Técnico - Dadosfera/Dados Consolidados/products_consolidado.parquet', index=False)
df_customers.to_parquet('/content/drive/MyDrive/Case Técnico - Dadosfera/Dados Consolidados/customers_consolidado.parquet', index=False)